### Data Preprocessing
You can use your own way of preprocessing to enhance results. Best results will lead to bonus points.

In [1]:
import pandas as pd
import tensorflow as tf
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [2]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
 35% 9.00M/25.7M [00:00<00:00, 34.2MB/s]
100% 25.7M/25.7M [00:00<00:00, 84.1MB/s]


In [3]:
!unzip imdb-dataset-of-50k-movie-reviews.zip

Archive:  imdb-dataset-of-50k-movie-reviews.zip
  inflating: IMDB Dataset.csv        


In [4]:
reviews = pd.read_csv("/content/IMDB Dataset.csv")

In [5]:
reviews.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [6]:
reviews.sentiment = reviews.sentiment.map({'positive':1,'negative':0})

In [7]:
reviews.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [8]:
reviews.isna().sum()

review       0
sentiment    0
dtype: int64

In [9]:
reviews.duplicated().sum()

418

In [10]:
reviews.drop_duplicates(inplace=True)

In [11]:
reviews.reset_index(drop=True, inplace=True)

In [12]:
reviews.duplicated().sum()

0

In [13]:
reviews.shape

(49582, 2)

In [14]:
reviews["length"] = reviews.review.str.len()

In [15]:
reviews.head()

,review,sentiment,length
0,One of the other reviewers has mentioned that ...,1,1761
1,A wonderful little production. <br /><br />The...,1,998
2,I thought this was a wonderful way to spend ti...,1,926
3,Basically there's a family where a little boy ...,0,748
4,"Petter Mattei's ""Love in the Time of Money"" is...",1,1317


In [16]:
reviews.describe()

,sentiment,length
count,49582.000000,49582.000000
mean,0.501876,1310.568230
std,0.500002,990.762238
min,0.000000,32.000000
25%,0.000000,699.000000
50%,1.000000,971.000000
75%,1.000000,1592.000000
max,1.000000,13704.000000


In [17]:

reviews_list = reviews.review.to_list()

In [18]:
len(reviews_list[0].split())

307

In [19]:
reviews.to_csv("clean.csv", index=False)

In [20]:
for i in range(len(reviews_list)):
    words = reviews_list[i].split()
    if len(words) > 200:
        reviews_list[i] = ' '.join(words[:200])

Crearing Tokenizer

In [21]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [22]:
tokenizer = Tokenizer()

In [23]:
len(reviews_list)

49582

In [24]:
tokenizer.fit_on_texts(reviews_list)

In [25]:
len(tokenizer.word_index)

103342

In [26]:
X = tokenizer.texts_to_sequences(reviews_list)

In [27]:
Y = reviews.sentiment.to_list()

In [28]:
len(X), len(Y)

(49582, 49582)

In [29]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [30]:
X = pad_sequences(X, maxlen=200, padding='pre')

In [31]:
from sklearn.model_selection import train_test_split
train_X, val_X, train_y, val_y = train_test_split(X, Y, test_size=0.2, random_state=1)

In [32]:
len(val_X), len(val_y), len(train_X), len(train_y)

(9917, 9917, 39665, 39665)

In [33]:
for i in train_X:
    if len(i) > 200:
        print(len(i))

DataSet Class

In [34]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, x_data, y_data, transform=None):
        self.reviews = x_data
        self.labels = y_data
        self.transform = transform

    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, idx):
      try:
        sample = {'review': self.reviews[idx], 'label': self.labels[idx]}
      except:
        sample = {'review': np.zeros_like(self.reviews[0]), 'label': 0}

      if self.transform:
          sample = self.transform(sample)

      return sample

### Sequence models
Following is an example code for simple LSTMs containing one layer. Your implementation should be generic in which user can be able to create multiple layers if required.

In [35]:

class LSTM(nn.Module):

    def __init__(self, input_dim, hidden_dim, vocab_size):
        super(LSTM, self).__init__()
        self.input_dim  = input_dim
        self.hidden_dim = hidden_dim

        self.embedding = nn.Embedding(vocab_size, input_dim)

        self.W_x = nn.Linear(input_dim, 4 * hidden_dim, bias=True)
        self.W_h = nn.Linear(hidden_dim, 4 * hidden_dim, bias=True)

        self.drop_layer = nn.Dropout(p=0.25)
        self.output_layer = nn.Linear(hidden_dim, 1)  # Output layer for binary classification

        self.Sigmoid = nn.Sigmoid()
        self.Tanh = nn.Tanh()
        self.h = None
        self.c = None

    def lstm_step(self, inp, prev_hidden_cell):
        h_prev, c_prev = prev_hidden_cell

        inp = self.embedding(inp)

        inp = self.drop_layer(inp)

        activation = self.W_x(inp) + self.W_h(h_prev)

        ai, af, ac, ao = activation.chunk(4, 1)

        in_gate = self.Sigmoid(ai)
        forget_gate = self.Sigmoid(af)
        cell_gate = self.Tanh(ac)
        out_gate = self.Sigmoid(ao)
        updated_c = forget_gate * c_prev + in_gate * cell_gate
        updated_h = out_gate * self.Tanh(updated_c)
        return updated_h, updated_c

    def forward(self, inp):
        batch_size, seq_len = inp.shape
        device = inp.device

        if self.h is None or self.c is None:
            h = torch.zeros(batch_size, self.hidden_dim).to(device)
            c = torch.zeros(batch_size, self.hidden_dim).to(device)
        else:
            h = self.h.detach().to(device)
            c = self.c.detach().to(device)

        for t in range(seq_len):
            x_t = inp[:, t]
            h, c = self.lstm_step(x_t, (h, c))

        self.h, self.c = h, c  # Update the internal states

        out = self.output_layer(h)
        return out

In [53]:
class GRU(nn.Module):
  def __init__(self, input_dim, hidden_dim, vocab_size):
    super(GRU, self).__init__()
    self.input_dim = input_dim
    self.hidden_dim = hidden_dim

    self.embedding = nn.Embedding(vocab_size, input_dim)
    self.W_x = nn.Linear(input_dim, 3 * hidden_dim, bias=True)
    self.W_h = nn.Linear(hidden_dim, 3 * hidden_dim, bias=True)

    # self.drop_layer = nn.Dropout(p=0.25)
    self.output_layer = nn.Linear(hidden_dim, 1)

    self.Sigmoid = nn.Sigmoid()
    self.Tanh = nn.Tanh()

  def gru_step(self, inp, prev_hidden):
    inp = self.embedding(inp)

    activation = self.W_x(inp) + self.W_h(prev_hidden)
    r, z, n = activation.chunk(3, 1)

    reset_gate = self.Sigmoid(r)
    update_gate = self.Sigmoid(z)
    new_gate = self.Tanh(n)

    updated_h = (1 - update_gate) * prev_hidden + update_gate * new_gate
    return updated_h

  def forward(self, inp):
    batch_size, seq_len = inp.shape
    device = inp.device
    h = torch.zeros(batch_size, self.hidden_dim).to(device)

    for t in range(seq_len):
      x_t = inp[:, t]
      h = self.gru_step(x_t, h)

    out = self.output_layer(h)
    return out

# Train

In [37]:
from tqdm import tqdm

In [38]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [39]:
device

device(type='cuda')

In [40]:
import torch
import numpy as np
from tqdm import tqdm

def train(model, num_epochs, criterion, optimizer, train_loader, val_loader, device):
    history = {'train_loss': [], 'val_loss': [], 'train_accuracy': [], 'val_accuracy': [], 'best_accuracy': -np.inf, 'min_loss': np.inf}

    for epoch in range(num_epochs):
        model.train()
        train_loss, train_corrects = 0, 0

        outer = tqdm(total=len(train_loader.dataset), desc=f'Train Epoch: {epoch + 1} / {num_epochs}')
        for batch in train_loader:
            try:
                inputs = batch['review'].to(device).long()
                labels = batch['label'].view(-1, 1).to(device).float()  # Convert labels to float

                optimizer.zero_grad()
                output = model(inputs)

                loss = criterion(output, labels)
                loss.backward()
                optimizer.step()

                train_loss += loss.item()

                # Use sigmoid to convert outputs to probabilities
                output_probs = torch.sigmoid(output)
                predicted = (output_probs > 0.5).float()  # Binary classification
                train_corrects += (predicted == labels).sum().item()

                outer.update(len(inputs))
            except Exception as e:
                print(f"Error: {e}")
                outer.update(len(inputs))

        outer.close()

        train_accuracy = 100. * train_corrects / len(train_loader.dataset)
        train_loss /= len(train_loader)
        print(f"Train Loss: {train_loss:.4f}")
        print(f"Train Accuracy: {train_accuracy:.2f}%")

        history['train_loss'].append(train_loss)
        history['train_accuracy'].append(train_accuracy)

        # Save the model state
        torch.save(model.state_dict(), f'model_epoch_{epoch+1}.pth')

        # Validation
        model.eval()
        val_loss, val_corrects = 0, 0

        with torch.no_grad():
            outer = tqdm(total=len(val_loader.dataset), desc='Validating')
            for batch in val_loader:
                inputs = batch['review'].to(device).long()
                labels = batch['label'].view(-1, 1).to(device).float()  # Convert labels to float

                output = model(inputs)
                loss = criterion(output, labels)

                val_loss += loss.item()

                # Use sigmoid to convert outputs to probabilities
                output_probs = torch.sigmoid(output)
                predicted = (output_probs > 0.5).float()  # Binary classification
                val_corrects += (predicted == labels).sum().item()

                outer.update(len(inputs))

            outer.close()

            val_accuracy = 100. * val_corrects / len(val_loader.dataset)
            val_loss /= len(val_loader)
            print(f"Val Loss: {val_loss:.4f}")
            print(f"Val Accuracy: {val_accuracy:.2f}%")

            history['val_loss'].append(val_loss)
            history['val_accuracy'].append(val_accuracy)

            if val_accuracy > history['best_accuracy']:
                history['best_accuracy'] = val_accuracy

            if val_loss < history['min_loss']:
                history['min_loss'] = val_loss

    return history

In [41]:
trainLoader = CustomDataset(train_X, train_y)
valLoader = CustomDataset(val_X, val_y)
# testLoader = CustomDataset(test_x, test_y, transform)

In [42]:
train_loader = torch.utils.data.DataLoader(trainLoader, batch_size=256, shuffle=True, drop_last = True)
val_loader = torch.utils.data.DataLoader(valLoader, batch_size=256, shuffle=True, drop_last= True)
# test_loader = torch.utils.data.DataLoader(testLoader, batch_size=32, shuffle=True)

In [43]:
criterion = nn.BCEWithLogitsLoss()
lstm_model = LSTM(200,256, 103343)

In [44]:
# import torch
# print(torch.cuda.is_available())
# print(torch.cuda.current_device())
# print(torch.cuda.get_device_name(torch.cuda.current_device()))


In [45]:
lstm_model.to(device)

LSTM(
  (embedding): Embedding(103343, 200)
  (W_x): Linear(in_features=200, out_features=1024, bias=True)
  (W_h): Linear(in_features=256, out_features=1024, bias=True)
  (drop_layer): Dropout(p=0.25, inplace=False)
  (output_layer): Linear(in_features=256, out_features=1, bias=True)
  (Sigmoid): Sigmoid()
  (Tanh): Tanh()
)

In [47]:
optimizer = torch.optim.Adam(lstm_model.parameters(), lr=0.001)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device
lstm_history = train(lstm_model, 10, criterion, optimizer, train_loader, val_loader, device)

device(type='cuda')

In [54]:
with open ("lstm_results.json", "w") as f:
    import json
    json.dump(lstm_history, f)

In [54]:
gru_model = GRU(200,128, 103343)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(gru_model.parameters(), lr=0.001)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
gru_model.to(device)

GRU(
  (embedding): Embedding(103343, 200)
  (W_x): Linear(in_features=200, out_features=384, bias=True)
  (W_h): Linear(in_features=128, out_features=384, bias=True)
  (output_layer): Linear(in_features=128, out_features=1, bias=True)
  (Sigmoid): Sigmoid()
  (Tanh): Tanh()
)

In [55]:
gru_history = train(gru_model, 10, criterion, optimizer, train_loader, val_loader, device)



Train Epoch: 1 / 10:   0%|          | 0/39665 [00:00<?, ?it/s]

Train Epoch: 1 / 10:   1%|          | 256/39665 [00:00<01:00, 649.13it/s]

Train Epoch: 1 / 10:   1%|▏         | 512/39665 [00:00<00:59, 660.06it/s]

Train Epoch: 1 / 10:   2%|▏         | 768/39665 [00:01<00:58, 666.21it/s]

Train Epoch: 1 / 10:   3%|▎         | 1024/39665 [00:01<00:57, 668.80it/s]

Train Epoch: 1 / 10:   3%|▎         | 1280/39665 [00:01<00:57, 670.45it/s]

Train Epoch: 1 / 10:   4%|▍         | 1536/39665 [00:02<00:57, 664.85it/s]

Train Epoch: 1 / 10:   5%|▍         | 1792/39665 [00:02<00:56, 664.80it/s]

Train Epoch: 1 / 10:   5%|▌         | 2048/39665 [00:03<00:56, 661.54it/s]

Train Epoch: 1 / 10:   6%|▌         | 2304/39665 [00:03<00:56, 665.27it/s]

Train Epoch: 1 / 10:   6%|▋         | 2560/39665 [00:03<00:55, 667.33it/s]

Train Epoch: 1 / 10:   7%|▋         | 2816/39665 [00:04<00:55, 668.99it/s]

Train Epoch: 1 / 10:   8%|▊         | 3072/39665 [00:04<00:54, 669.60it/s]

Train Epoch: 1 / 10:   8%

Train Loss: 0.5920
Train Accuracy: 67.43%




Validating:   0%|          | 0/9917 [00:00<?, ?it/s]

Validating:   5%|▌         | 512/9917 [00:00<00:02, 3341.47it/s]

Validating:  10%|█         | 1024/9917 [00:00<00:02, 3383.45it/s]

Validating:  15%|█▌        | 1536/9917 [00:00<00:02, 3406.45it/s]

Validating:  21%|██        | 2048/9917 [00:00<00:02, 3386.12it/s]

Validating:  26%|██▌       | 2560/9917 [00:00<00:02, 3427.85it/s]

Validating:  31%|███       | 3072/9917 [00:00<00:01, 3544.72it/s]

Validating:  36%|███▌      | 3584/9917 [00:01<00:01, 3324.79it/s]

Validating:  41%|████▏     | 4096/9917 [00:01<00:01, 3226.69it/s]

Validating:  46%|████▋     | 4608/9917 [00:01<00:01, 3213.50it/s]

Validating:  52%|█████▏    | 5120/9917 [00:01<00:01, 3163.26it/s]

Validating:  57%|█████▋    | 5632/9917 [00:01<00:01, 3044.00it/s]

Validating:  62%|██████▏   | 6144/9917 [00:01<00:01, 3058.65it/s]

Validating:  67%|██████▋   | 6656/9917 [00:02<00:01, 2889.38it/s]

Validating:  72%|███████▏  | 7168/9917 [00:02<00:00, 2884.61it/s]

Validat

Val Loss: 0.6162
Val Accuracy: 64.41%




Train Epoch: 2 / 10:   0%|          | 0/39665 [00:00<?, ?it/s]

Train Epoch: 2 / 10:   1%|          | 256/39665 [00:00<01:08, 579.21it/s]

Train Epoch: 2 / 10:   1%|▏         | 512/39665 [00:00<01:02, 627.31it/s]

Train Epoch: 2 / 10:   2%|▏         | 768/39665 [00:01<01:00, 647.05it/s]

Train Epoch: 2 / 10:   3%|▎         | 1024/39665 [00:01<00:58, 657.89it/s]

Train Epoch: 2 / 10:   3%|▎         | 1280/39665 [00:01<00:58, 661.21it/s]

Train Epoch: 2 / 10:   4%|▍         | 1536/39665 [00:02<00:57, 660.71it/s]

Train Epoch: 2 / 10:   5%|▍         | 1792/39665 [00:02<00:57, 662.34it/s]

Train Epoch: 2 / 10:   5%|▌         | 2048/39665 [00:03<00:56, 664.66it/s]

Train Epoch: 2 / 10:   6%|▌         | 2304/39665 [00:03<00:56, 664.34it/s]

Train Epoch: 2 / 10:   6%|▋         | 2560/39665 [00:03<00:55, 663.04it/s]

Train Epoch: 2 / 10:   7%|▋         | 2816/39665 [00:04<00:55, 663.77it/s]

Train Epoch: 2 / 10:   8%|▊         | 3072/39665 [00:04<00:55, 664.94it/s]

Train Epoch: 2 / 10:   8%

...

Val Loss: 0.5317
Val Accuracy: 84.23%




Train Epoch: 10 / 10:   0%|          | 0/39665 [00:00<?, ?it/s]

Train Epoch: 10 / 10:   1%|          | 256/39665 [00:00<01:05, 603.00it/s]

Train Epoch: 10 / 10:   1%|▏         | 512/39665 [00:00<01:02, 624.94it/s]

Train Epoch: 10 / 10:   2%|▏         | 768/39665 [00:01<01:02, 621.65it/s]

Train Epoch: 10 / 10:   3%|▎         | 1024/39665 [00:01<01:02, 614.04it/s]

Train Epoch: 10 / 10:   3%|▎         | 1280/39665 [00:02<01:02, 611.66it/s]

Train Epoch: 10 / 10:   4%|▍         | 1536/39665 [00:02<01:02, 613.46it/s]

Train Epoch: 10 / 10:   5%|▍         | 1792/39665 [00:02<01:01, 612.15it/s]

Train Epoch: 10 / 10:   5%|▌         | 2048/39665 [00:03<01:01, 614.76it/s]

Train Epoch: 10 / 10:   6%|▌         | 2304/39665 [00:03<00:59, 629.12it/s]

Train Epoch: 10 / 10:   6%|▋         | 2560/39665 [00:04<00:57, 639.81it/s]

Train Epoch: 10 / 10:   7%|▋         | 2816/39665 [00:04<00:56, 647.29it/s]

Train Epoch: 10 / 10:   8%|▊         | 3072/39665 [00:04<00:56, 651.92it/s]

Train Epoch:

Train Loss: 0.0300
Train Accuracy: 98.62%




Validating:   0%|          | 0/9917 [00:00<?, ?it/s]

Validating:   5%|▌         | 512/9917 [00:00<00:02, 3715.22it/s]

Validating:  10%|█         | 1024/9917 [00:00<00:02, 4147.36it/s]

Validating:  15%|█▌        | 1536/9917 [00:00<00:01, 4335.88it/s]

Validating:  21%|██        | 2048/9917 [00:00<00:01, 4481.14it/s]

Validating:  26%|██▌       | 2560/9917 [00:00<00:01, 4522.29it/s]

Validating:  31%|███       | 3072/9917 [00:00<00:01, 4607.95it/s]

Validating:  36%|███▌      | 3584/9917 [00:00<00:01, 4631.32it/s]

Validating:  41%|████▏     | 4096/9917 [00:00<00:01, 4653.42it/s]

Validating:  46%|████▋     | 4608/9917 [00:01<00:01, 4545.66it/s]

Validating:  52%|█████▏    | 5120/9917 [00:01<00:01, 4262.07it/s]

Validating:  57%|█████▋    | 5632/9917 [00:01<00:00, 4410.02it/s]

Validating:  62%|██████▏   | 6144/9917 [00:01<00:00, 4486.67it/s]

Validating:  67%|██████▋   | 6656/9917 [00:01<00:00, 4541.82it/s]

Validating:  72%|███████▏  | 7168/9917 [00:01<00:00, 4558.11it/s]

Validat

Val Loss: 0.6118
Val Accuracy: 83.70%


# Testing

In [60]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GRU(200,128, 103343)
# model = LSTM(200,256, 103343) #For LSTM
model.load_state_dict(torch.load('/content/model_epoch_5.pth')) # Enter Model's path
model.to(device)

GRU(
  (embedding): Embedding(103343, 200)
  (W_x): Linear(in_features=200, out_features=384, bias=True)
  (W_h): Linear(in_features=128, out_features=384, bias=True)
  (output_layer): Linear(in_features=128, out_features=1, bias=True)
  (Sigmoid): Sigmoid()
  (Tanh): Tanh()
)

In [62]:
inp = input("Enter review: ")
inp = tokenizer.texts_to_sequences([inp])
inp = pad_sequences(inp, maxlen=200, padding='pre')
inp = torch.tensor(inp).to(device)
output = model(inp)
output_probs = torch.sigmoid(output)
predicted = (output_probs > 0.5).float()
print(output_probs)
if predicted == 1:
    print("Positive")
else:
    print("Negative")

Enter review: Intro was good, but movie was below average, i would say a waste of money
tensor([[0.0101]], device='cuda:0', grad_fn=<SigmoidBackward0>)
Negative
